In [ ]:
import numpy as np
from scipy.stats import multivariate_normal as N
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd

def plot_gaussian_mixture(mu, cov, P, X = None, granularity = 50):
    
    if len(mu[0]) > 2:
        raise Exception("Plotting supported only for 2D data.")
    if len(mu) != len(cov):
        raise Exception("mu and cov don't have same length")
    if len(mu) != len(P):
        raise Exception("mu and P don't have same length")
    
    # set up mixted density function (slide 16)
    f = lambda x: sum([N.pdf(x, mu[i], cov[i]) * P[i]  for i in range(len(mu))])
          
    # create grid
    min_x1, max_x1 = np.inf, -np.inf
    min_x2, max_x2 = np.inf, -np.inf
    span = 5
    for i in range(len(mu)):
        min_x1 = min(min_x1, mu[i][0] - span * cov[i][0,0])
        min_x2 = min(min_x2, mu[i][1] - span * cov[i][1,1])
        max_x1 = max(max_x1, mu[i][0] + span * cov[i][0,0])
        max_x2 = max(max_x2, mu[i][1] + span * cov[i][1,1])
    min_x1 -= 1
    min_x2 -= 1
    max_x1 += 1
    max_x2 += 1
    X1 = np.linspace(min_x1, max_x1, granularity)
    X2 = np.linspace(min_x2, max_x2, granularity)
    Z = np.zeros((granularity, granularity))
    
    # compute densities in grid
    M_X1, M_X2 = np.meshgrid(X1, X2)
    for i in range(granularity):
        for j in range(granularity):
            x1, x2 = M_X1[i,j], M_X2[i,j]
            Z[i,j] = f([x1,x2])
    
    # create 3D plot with surface
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 10))
    ax.plot_wireframe(M_X1, M_X2, Z, linewidth=1,alpha=0.6)
    ax.set_xlim(min_x1, max_x1)
    ax.set_ylim(min_x2, max_x2)
    ax.set_xlabel("X1")
    ax.set_ylabel("X2")
    
    # scatter the points into the plot
    if X is not None:
        ax.scatter(X[:,0], X[:,1], 0)

# Exercise 2

## Plots
### Dependent

In [ ]:
%matplotlib notebook
Iris_pca = pd.read_csv("iris_pca.csv").values
mu_init = [np.array(mu) for mu in [[-3.59, 0.25], [-1.09, -0.46], [0.75, 1.07]]]
cov_init = [np.eye(2) for i in range(3)]
probs, mu, cov, P = EM(Iris_pca, 3, 0.001, mu=mu_init, cov=cov_init, independent=False)
print(np.round(mu, 2), np.round(cov, 2), np.round(P, 2))
plot_gaussian_mixture(mu, cov, P, Iris_pca)

### Independent

In [ ]:
%matplotlib notebook
Iris_pca = pd.read_csv("iris_pca.csv").values
mu_init = [np.array(mu) for mu in [[-3.59, 0.25], [-1.09, -0.46], [0.75, 1.07]]]
cov_init = [np.eye(2) for i in range(3)]
probs, mu, cov, P = EM(Iris_pca, 3, 0.001, mu=mu_init, cov=cov_init, independent=True)
print(np.round(mu, 2), np.round(cov, 2), np.round(P, 2))
plot_gaussian_mixture(mu, cov, P, Iris_pca)